# Ledger Data Types 

This file defines a Python implementation that simulates the ledger data types and their interfaces as defined in **Build > Reference documentation > The Compact language > Ledger data types**. 

## Misc 

In [201]:
from typing import TypeVar, Generic
from typing import Dict

### Compact Types

We use the following type aliases since Compact features finite precision integers. For now, we'll map them to Python's `int` type, but we could change these in the future to simulate overflows etc... 

uint64 = int 
uint16 = int

These are defined in `std.compact`. **TODO**: inhabit with a type representing their actual definition

In [202]:
CoinInfo = ()
QualifiedCoinInfo = ()
ZswapCoinPublicKey = ()
ContractAddress = ()

"""
According to the docs: 

Writes a CoinInfo to this Cell, which is transformed into a QualifiedCoinInfo 
at runtime by looking up the relevant Merkle tree index. This index must have 
been allocated within the current transaction or this write fails

"""
def qualify(ci:CoinInfo, recepient:Recipient) -> QualifiedCoinInfo: 
    # TODO: implement 
    return ()

Wrapper class for values that hold a value of type `ZswapCoinPublicKey` or `ContractAddress`. This is a special case of Compact's `Either` type, which I'm not sure how to define generically in Python

**TODO**: Perhaps for this (and potentially more, such as default values or subtyping/casting) reasons it's better to not allow arbitrary python types when constructing generic ledger ADTs, but rather embed Compact types using some kind of class hierarchy. 

In [203]:
class Recipient: 
    
    _which_value:bool 
    _pubkey_value:ZswapCoinPublicKey 
    _addr_value:ContractAddress

    def __init__(self, which : bool, value) -> None: 
        self._which_value = which
        if which: 
            self._pubkey_value = value 
        else:
            self._addr_value = value 

    def read_pubkey(self) -> ZswapCoinPublicKey: 
        if self._which_value: 
            return self._pubkey_value
        else: 
            raise Exception("Tried to read left choice of a sum but the value was right")

    def read_addr(self) -> ContractAddress: 
        if not self.which_value: 
            return self._addr_value
        else: 
            raise Exception("Tried to read right choice of a sum but the value was left")

    def which(self) -> bool: 
        return sefl._which_value

### Other

The following represents any kind of run-time failure that might occur while executing a piece of compact code

In [204]:
class RuntimeError(Exception):
    pass

## Counter

In [226]:
class MK_Counter: 
    _counter:uint64

    def __init__(self) -> None: 
        self._counter = 0

    # Decrement the counter by the given amount, or throw a run-time error if that would bring the counter below 0
    def decrement(self , amount : uint16) -> None: # amount has 16 bit precision 
        if self.counter >= amount: 
            self._counter -= amount
        else: 
            raise RunTimeError(f'Cannot decrement counter value {self._counter} with amount {amount}')

    # Increment the counter by the given amount
    def increment(self , amount : uint16) -> None: # amount has 16 bit precision
        self._counter += amount     

    # Check if the counter is smaller than a given threshold
    def less_than(self, threshold : int64) -> bool:
        return self._counter < threshold

    def read(self) -> uint64: 
        return self._counter 

    def reset_to_default(self) -> None: 
        self._counter = 0

## Cell 



Stores a value of any type

In [228]:
T_Cell = TypeVar('T_Cell')

class MK_Cell(Generic[T_Cell]): 

    # The current resp. default value of the type stored in the cell
    _value         : T_Cell 
    _default_value : T_Cell

    # Initializes the cell. Creation of a new cell requires us to pass in a default value for the 
    # type that's stored in the cell. Since all types in Compact have a default value, and `T_Cell` 
    # should be a Compact type, this should always exist
    def __init__(self, default : T_Cell) -> None: 
        self._value = default
        self._default_value = default

    def read(self) -> T_Cell: 
        return self._value

    def reset_to_default(self) -> None:
        self._value = self._default_value

    def write(self, new_value : T_Cell) -> None: 
        self._value = new_value 

    def write_coin(self, ci : CoinInfo, recepient : Recipient) -> None:
        
        # TODO: validate that this check actually works 
        if not isinstance(self, MK_Cell[QualifiedCoinInfo]):
            raise RuntimeException(f'Tried to invoke the write_coin operation on a cell with the wrong type. Value: {self._value}')

        self._value = qualify(ci, recipient)

## Set

Based on Python's `Set`

In [295]:
T_Set = TypeVar('T_Set')

class MK_Set(Set[T_Set]): 

    def is_empty(self) -> bool: 
        return self.len() == 0
    
    def member(self, elem : T_Set) -> bool: 
        return elem in self

    def remove(self, elem : T_Set) -> None: 
        self.remove(elem)

    def reset_to_default(self) -> None: 
        self.clear()

    def size(self) -> uint64: 
        return self.len()

    def insert(self, elem : T_Set) -> None: 
        self.add(elem)
        
    def insert_coin(self, ci : CoinInfo, recepient : Recipient) -> None: 
        if not isinstance(self, MK_Set[QualifiedCoinInfo]):
            raise RuntimeException(f'Tried to invoke the insert_coin operation on a set that stores values of a different type. Values: {self._values}')
        self.add(qualify(ci , recepient))

## Map

Based on Python's `Dict`

In [297]:
T_Key = TypeVar('T_Key')
T_Val = TypeVar('T_Key') 

class MK_Map(dict[T_Key,T_Val]): 

    _default_value : T_Val 

    def __init__(self , default : T_Val) -> None: 
        self._default_value = default
        super(MK_Map, self).__init__()
    
    def insert(self, k : T_Key, v : T_Val) -> None: 
        self[k] = v

    def insert_coin(self, k : T_Key, ci : CoinInfo, recipient : Recipient) -> None: 
        if not isinstance(self, MK_Map[T_Key,QualifiedCoinInfo]):
            raise RunTimeException(f'Tried to invoke the insert_coin operation on a map that stores values of a different type. Values:{self}')

    def insert_default(self, k : T_Key) -> None: 
        self[k] = _default_value

    # The empty dictionary evaluates to `False`, apparently ... 
    def is_empty(self) -> bool: 
        return not bool(self)

    def lookup(self, k : T_Key) -> T_Val: 
        return self[k]

    def member(self, k : T_Key) -> bool: 
        return k in self

    def remove(self, k : T_Key) -> bool: 
        del self[k]

    def reset_to_default(self) -> None:
        self.clear()

    def size(self) -> None: 
        self.len()

    


## List

Based on Python's `List`

In [296]:
# TODO